In [25]:
import pandas as pd
import re
import statistics
import nltk
from nltk.tag import pos_tag
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/limjohn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/limjohn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/limjohn/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

This is meant to find content based features. Each feature will require its own preprocessing

In [26]:
df = pd.read_csv("./data/WELFake_Dataset.csv")
print(df.columns)
print(df.shape)
print(type(df.loc[7, "text"]))
print(df)

text_iterator = df["text"]
title_iterator = df["title"]
def combine_columns(row):
    return str(row["title"]) + "." + str(row["text"])


# Apply the custom function to create a new column 'Combined'
df["Combined"] = df.apply(combine_columns, axis=1)
merged_iterator = df["Combined"]

Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')
(72134, 4)
<class 'str'>
       Unnamed: 0                                              title  \
0               0  LAW ENFORCEMENT ON HIGH ALERT Following Threat...   
1               1                                                NaN   
2               2  UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...   
3               3  Bobby Jindal, raised Hindu, uses story of Chri...   
4               4  SATAN 2: Russia unvelis an image of its terrif...   
...           ...                                                ...   
72129       72129  Russians steal research on Trump in hack of U....   
72130       72130   WATCH: Giuliani Demands That Democrats Apolog...   
72131       72131  Migrants Refuse To Leave Train At Refugee Camp...   
72132       72132  Trump tussle gives unpopular Mexican leader mu...   
72133       72133  Goldman Sachs Endorses Hillary Clinton For Pre...   

                                              

Tokenise sentence length for sentence length, keep stop word

In [27]:
def process_text(text_iterator):
    text_processed = []
    empty_indexes = []
    for index,text in enumerate(text_iterator):    
        try:
            # remove some supers 
            t = re.sub("[^a-zA-Z0-9\.\?\!]", " ", text)
            # remove multiple spaces
            t = re.sub(r" +", " ", t)
            # remove newline
            t = re.sub(r"\n", " ", t)
            # clear trailing whitespaces
            t = t.strip()
            # lowercase
            t = t.lower()
            # tokenise sentences
            t = re.split("\!|\.|\?",t)
            # strip sentence
            t = [item.strip() for item in t]
            # drop empty string 
            t = list(filter(lambda x: x != "", t))
            # tokenise words 
            sentences_tokenised = []
            for sentence in t: 
                #tokenise
                s = sentence.split(" ")
                sentences_tokenised.append(s)                                
            if len(sentences_tokenised) == 0:
                print(text)
                empty_indexes.append(index)
                text_processed.append(None)
            else:
                text_processed.append(sentences_tokenised)
        except Exception as e:
            print(e)
            text_processed.append(None)
            continue
        
    return text_processed

print(process_text(["this is a test. um oops! not really? haha haha!","thats the rub. whats a rub?? ruba# dub dub!"]))


[[['this', 'is', 'a', 'test'], ['um', 'oops'], ['not', 'really'], ['haha', 'haha']], [['thats', 'the', 'rub'], ['whats', 'a', 'rub'], ['ruba', 'dub', 'dub']]]


In [28]:
merged_processed = process_text(merged_iterator)
print(len(merged_processed))
print(merged_iterator[0])
print(merged_processed[0])
print(df["label"][0])

Чем закончатся изменения региональной динамики на Ближнем Востоке | Новое восточное обозрение.Как отмечает в своей новой статье ветеран боевых действий США Гордон Даф, сегодня в «большой региональной игре» стороны меняют свою приверженность настолько же быстро, насколько заключают новые союзы. Так, Турция и Саудовская Аравия планируют усилить поддержку ДАИШ, пока Вашингтон занимает неопределенную позицию по этому вопросу. Тель-Авив, со своей стороны, посылает недвусмысленные сигналы о том, что все контакты с Белым домом «отныне нужно держать через него», на что уже, судя по всему, отреагировал Иран, который помнит обещания Трампа об отмене договора по иранской ядерной программе. После того, как боевики ХАМАС начали переправляться ЦРУ в лагеря подготовки ДАИШ, практически ни одна из региональных держав публично не возражает против дальнейшей оккупации Израилем палестинских территорий. В условиях, когда роль США сокращается в регионе, а Анкара и Тель-Авив значительно усиливают свои позиц

In [41]:
# Count words per sentence, long short sentence
def sentence_count(row, limit):
    lengths = []
    short_sentence = 0
    long_sentence = 0
    sentences = len(row)
    for sentence in row:
        length = len(sentence)
        lengths.append(length)
        if length <= limit:
            short_sentence += 1
        else:
            long_sentence += 1
    if len(lengths) > 1:
        return (
            statistics.median(lengths),
            statistics.variance(lengths),
            short_sentence,
            long_sentence,
            sentences,
        )
    else:
        return lengths[0], 0, short_sentence, long_sentence, sentences


# Short/Long Sentences
limit = 10


words_per_sentence_ls = []
short_sentence_ls = []
long_sentence_ls = []
sentences_len_ls = [] 
for i in merged_processed:
    try:
        (
            median_sentence_length,
            var_sentence_length,
            short_sentences,
            long_sentences,
            sentences,
        ) = sentence_count(i, limit)
        words_per_sentence_ls.append((median_sentence_length, var_sentence_length))
        short_sentence_ls.append(short_sentences)
        long_sentence_ls.append(long_sentences)
        sentences_len_ls.append(sentences)
    except:
        words_per_sentence_ls.append((0, 0))
        short_sentence_ls.append(0)
        long_sentence_ls.append(0)
        sentences_len_ls.append(0)
        continue



In [30]:
print(len(words_per_sentence_ls))
print(len(long_sentence_ls))
print(len(short_sentence_ls))
print(len(sentences_len_ls))

72134
72134
72134
72134


Generate parts of speech - drop stop words. 


In [31]:
# tokenise per document
def process_text(text_iterator):
    text_processed = []
    for text in text_iterator:        
        try:
            # remove punctuation
            t = re.sub("[^a-zA-Z0-9]", " ", text)
            # remove multiple spaces
            t = re.sub(r" +", " ", t)
            # remove newline
            t = re.sub(r"\n", " ", t)
            # clear trailing whitespaces
            t = t.strip()
            # lowercase
            t = t.lower()
            # tokenise
            t = t.split(" ")
            # drop empty string
            t = list(filter(lambda x: x != "", t))
            if len(t) == 0:
                text_processed.append(None)
            else:
                text_processed.append(t)
        except:
            text_processed.append(None)
            continue
    return text_processed

text_processed = process_text(text_iterator)
title_processed = process_text(title_iterator)

print(len(text_processed))
print(len(title_processed))
print(text_processed[2])

stop_words = set(stopwords.words("english"))

def remove_stop_words_inner(text, stop_words):
    no_stop_words = []
    if text is None:
        return None
    for token in text:
        if token not in stop_words:
            no_stop_words.append(token)
    return no_stop_words

def remove_stop_words_outer(text_list, stop_words):
    for index, text in enumerate(text_list):
        t = remove_stop_words_inner(text, stop_words)
        text_list[index] = t
    return text_list


text_no_stop_words = remove_stop_words_outer(text_processed, stop_words)
title_no_stop_words = remove_stop_words_outer(title_processed, stop_words)
print(text_no_stop_words[2])
print(len(text_no_stop_words))
print(len(title_no_stop_words))

72134
72134
['now', 'most', 'of', 'the', 'demonstrators', 'gathered', 'last', 'night', 'were', 'exercising', 'their', 'constitutional', 'and', 'protected', 'right', 'to', 'peaceful', 'protest', 'in', 'order', 'to', 'raise', 'issues', 'and', 'create', 'change', 'loretta', 'lynch', 'aka', 'eric', 'holder', 'in', 'a', 'skirt']
['demonstrators', 'gathered', 'last', 'night', 'exercising', 'constitutional', 'protected', 'right', 'peaceful', 'protest', 'order', 'raise', 'issues', 'create', 'change', 'loretta', 'lynch', 'aka', 'eric', 'holder', 'skirt']
72134
72134


Merge Title and Text

In [32]:
merged_title_text = []
for i,v in enumerate(title_no_stop_words):
    if (v == None):    
        merge = text_no_stop_words[i]
    elif (text_no_stop_words[i]==None):
        merge = v
    else:
        merge = v + text_no_stop_words[i]
    merged_title_text.append(merge)
    
print(len(merged_title_text))

72134


In [33]:
# parts of speech per document
pos_merged =[]
for doc in merged_title_text:
    try: 
        pos_merged.append(pos_tag(doc))
    except:
        pos_merged.append([])
print(len(pos_merged))

72134


In [34]:
print(pos_merged[0])
print(len(pos_merged[0]))
print(len(pos_merged))

[('law', 'NN'), ('enforcement', 'NN'), ('high', 'JJ'), ('alert', 'NN'), ('following', 'VBG'), ('threats', 'NNS'), ('cops', 'VBP'), ('whites', 'VBZ'), ('9', 'CD'), ('11by', 'CD'), ('blacklivesmatter', 'NN'), ('fyf911', 'NN'), ('terrorists', 'NNS'), ('video', 'VBP'), ('comment', 'NN'), ('expected', 'VBN'), ('barack', 'RB'), ('obama', 'JJ'), ('members', 'NNS'), ('fyf911', 'VBP'), ('fukyoflag', 'JJ'), ('blacklivesmatter', 'NN'), ('movements', 'NNS'), ('called', 'VBD'), ('lynching', 'VBG'), ('hanging', 'VBG'), ('white', 'JJ'), ('people', 'NNS'), ('cops', 'NNS'), ('encouraged', 'VBD'), ('others', 'NNS'), ('radio', 'VB'), ('show', 'NN'), ('tuesday', 'JJ'), ('night', 'NN'), ('turn', 'VB'), ('tide', 'JJ'), ('kill', 'NNP'), ('white', 'JJ'), ('people', 'NNS'), ('cops', 'NNS'), ('send', 'VBP'), ('message', 'NN'), ('killing', 'VBG'), ('black', 'JJ'), ('people', 'NNS'), ('america', 'VBP'), ('one', 'CD'), ('f', 'NN'), ('yoflag', 'NN'), ('organizers', 'NNS'), ('called', 'VBD'), ('sunshine', 'NN'), ('r

In [35]:
# count number of adverbs 
# CV 
adverbs  = ['RB','RBR','RBS','WRB']
nouns = ["NN","NNS"]
proper_nouns = ["NNP","NNPS"]
adjective = ["JJ","JJR","JJS"]

def count_word_pos(doc, word_type):
    count = 0
    for word in doc:
        if word[1] in word_type:
            count+=1 
    return count
            
adverb_raw_count = [count_word_pos(doc,adverbs) for doc in pos_merged]
nouns_raw_count = [count_word_pos(doc,nouns) for doc in pos_merged]
proper_nouns_raw_count = [count_word_pos(doc,proper_nouns) for doc in pos_merged]
adjective_raw_count = [count_word_pos(doc,adjective) for doc in pos_merged]
print(len(adverb_raw_count))
print(len(nouns_raw_count))
print(len(proper_nouns_raw_count))
print(len(adjective_raw_count))

72134
72134
72134
72134


In [42]:
print(adverb_raw_count[0])
# create table 
print(df.columns)
df_pos = pd.DataFrame()
df_pos["label"] = df["label"]
df_pos['adverbs'] = adverb_raw_count
df_pos['nouns'] = nouns_raw_count
df_pos['proper_nouns'] = proper_nouns_raw_count
df_pos['adjective'] = adjective_raw_count

# Generate df containing the words per sentence vector
df_pos["words_per_sentence"] = words_per_sentence_ls
df_pos["num_long_sentences"] = long_sentence_ls
df_pos["num_short_sentences"] = short_sentence_ls
df_pos["num_of_sentences"] = sentences_len_ls

18
Index(['Unnamed: 0', 'title', 'text', 'label', 'Combined'], dtype='object')


In [43]:
print(df_pos.shape)
print(df_pos.columns)
df_pos.to_csv("./data/content_features_document.csv")
df_pos.to_pickle("./data/content_features_document")

(72134, 9)
Index(['label', 'adverbs', 'nouns', 'proper_nouns', 'adjective',
       'words_per_sentence', 'num_long_sentences', 'num_short_sentences',
       'num_of_sentences'],
      dtype='object')
